In [6]:
import gensim
import pickle as pkl
import time

In [11]:
# Load model from Google .bin and pickle
# model = gensim.models.KeyedVectors.load_word2vec_format('./data/GoogleNews-vectors-negative300.bin', binary=True)
# with open("./data/model.pkl", "wb") as f:
#     pkl.dump(model, f)

In [12]:
# Load model from pickle
t0 = time.time()
with open("./data/model.pkl", "rb") as f:
    model = pkl.load(f)
print("Model loaded in", time.time() - t0, "seconds")

Model loaded in 8.979926347732544 seconds
